In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 19.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.preprocessing.sequence import TimeseriesGenerator
from transformers import BertTokenizer, BertModel

In [ ]:
from google.colab import files

In [ ]:
df1 = files.upload()

Saving SBIN.csv to SBIN.csv


In [ ]:
df2 = files.upload()

Saving Tweets.csv to Tweets.csv


In [ ]:
# Load the stock price dataset
sbin_df = pd.read_csv('SBIN.csv')

# Load the tweets dataset
tweets_df = pd.read_csv('Tweets.csv')

In [ ]:
# Perform sentiment analysis using TextBlob
sentiment_scores = []
for tweet in tweets_df['Tweets']:
    analysis = TextBlob(tweet)
    sentiment_scores.append(analysis.sentiment.polarity)
tweets_df['Sentiment'] = sentiment_scores

In [ ]:
# Merge the stock price and tweets datasets based on date
merged_df = pd.merge(sbin_df, tweets_df, on='Date', how='inner')

In [ ]:
print(merged_df['Close'])

Series([], Name: Close, dtype: float64)


In [ ]:
X = merged_df[['Open', 'High', 'Low', 'Volume', 'Sentiment']].values
y = merged_df['Close'].values

(0,)

In [ ]:
# Prepare data for LSTM
scaler = MinMaxScaler()
X = merged_df[['Open', 'High', 'Low', 'Volume', 'Sentiment']].values
y = merged_df['Close'].values
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ValueError: ignored

In [ ]:
# Generate time series data for LSTM
time_steps = 10
train_generator = TimeseriesGenerator(X_train, y_train, length=time_steps, batch_size=32)
test_generator = TimeseriesGenerator(X_test, y_test, length=time_steps, batch_size=32)

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, input_shape=(time_steps, X.shape[1])))
model.add(Dense(units=1, activation='linear'))
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the LSTM model
model.fit(train_generator, epochs=10)

In [ ]:
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# Tokenize the tweets
tokens = []
for tweet in tweets_df['Tweets']:
    encoded = tokenizer.encode(tweet, add_special_tokens=True)
    tokens.append(encoded)

# Pad the tokenized sequences to a fixed length
max_length = max([len(token) for token in tokens])
padded_tokens = np.array([token + [0]*(max_length - len(token)) for token in tokens])

In [ ]:
# Get the BERT embeddings for the tokenized sequences
bert_embeddings = []
for padded_token in padded_tokens:
    input_ids = torch.tensor([padded_token])
    with torch.no_grad():
        outputs = bert_model(input_ids)
        embeddings = outputs[0].numpy()
        bert_embeddings.append(embeddings.squeeze(0))

In [ ]:
# Convert the BERT embeddings to numpy arrays
X_bert_train = np.array(bert_embeddings[:len(X_train)])
X_bert_test = np.array(bert_embeddings[len(X_train):])

In [ ]:
# Build the LSTM model with BERT embeddings
model_bert = Sequential()
model_bert.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, input_shape=(X_bert_train.shape[1], X_bert_train.shape[2])))
model_bert.add(Dense(units=1, activation='linear'))
model_bert.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the LSTM model with BERT embeddings
model_bert.fit(X_bert_train, y_train, epochs=10, batch_size=32)

In [ ]:
# Evaluate the LSTM model on test data
mse = model.evaluate(test_generator)
print("MSE (LSTM):", mse)

In [ ]:
# Evaluate the LSTM model with BERT embeddings on test data
mse_bert = model_bert.evaluate(X_bert_test, y_test)
print("MSE (LSTM with BERT):", mse_bert)

In [ ]:
# Make predictions using the LSTM model
y_pred = model.predict(test_generator)
y_pred = scaler.inverse_transform(y_pred)

In [ ]:
# Make predictions using the LSTM model with BERT embeddings
y_pred_bert = model_bert.predict(X_bert_test)
y_pred_bert = scaler.inverse_transform(y_pred_bert)

In [ ]:
# Print the predicted stock prices
print("Predicted Stock Prices (LSTM):")
print(y_pred.flatten())

# Print the predicted stock prices with BERT embeddings
print("Predicted Stock Prices (LSTM with BERT):")
print(y_pred_bert.flatten())